In [ ]:
!pip install ccxt
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install TA-Lib


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import ccxt
import talib

In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from talib import RSI, BBANDS, MACD

# Load the historical BTC/USD data
btc_usd_data = pd.read_csv('/content/BTC-USD.csv')  # Change this to the path where you stored BTC-USD.csv

# Convert the 'Date' column to datetime format
btc_usd_data['Date'] = pd.to_datetime(btc_usd_data['Date'])

# Drop the 'Adj Close' column
btc_usd_data.drop('Adj Close', axis=1, inplace=True)

# Calculate 24hr Close Change
btc_usd_data['24hr_Close_Change'] = btc_usd_data['Close'].diff() / btc_usd_data['Close'].shift(1) * 100

# Market Direction based on Close price
btc_usd_data['Market_Direction'] = 0
btc_usd_data.loc[btc_usd_data['Close'] > btc_usd_data['Close'].shift(1), 'Market_Direction'] = 1
btc_usd_data.loc[btc_usd_data['Close'] < btc_usd_data['Close'].shift(1), 'Market_Direction'] = -1

# Calculate Volume Change
btc_usd_data['Volume_Change'] = btc_usd_data['Volume'].diff() / btc_usd_data['Volume'].shift(1) * 100

# Calculate RSI
btc_usd_data['RSI'] = RSI(btc_usd_data['Close'], timeperiod=14)

# Calculate Bollinger Bands
upper, middle, lower = BBANDS(btc_usd_data['Close'], timeperiod=20)
btc_usd_data['upper_band'] = upper
btc_usd_data['middle_band'] = middle
btc_usd_data['lower_band'] = lower

# Calculate MACD
macd, signal, _ = MACD(btc_usd_data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
btc_usd_data['MACD'] = macd
btc_usd_data['Signal_Line'] = signal

# Handle NaN values
btc_usd_data.fillna(0, inplace=True)

# Normalize columns
scaler = MinMaxScaler()
btc_usd_data[['Open', 'High', 'Low', 'Close', 'Volume']] = scaler.fit_transform(btc_usd_data[['Open', 'High', 'Low', 'Close', 'Volume']])

# Save the modified DataFrame
btc_usd_data.to_csv('/content/BTC-USD_new_with_indicators.csv', index=False)  # Change this to the path where you want to save the new CSV file


In [5]:
# Load and preprocess data
data = pd.read_csv('/content/BTC-USD_new_with_indicators.csv')
data = data.dropna()

features = ['Close', 'RSI', 'upper_band', 'middle_band', 'lower_band', 'MACD', 'Signal_Line']  # Change this line accordingly
X = data[features]
y = data['Market_Direction']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [6]:
# Define DQN model
class DQN(tf.keras.Model):
    def __init__(self, state_shape, action_shape):
        super(DQN, self).__init__()
        self.layer1 = tf.keras.layers.Dense(64, activation='relu', input_shape=state_shape)
        self.layer2 = tf.keras.layers.Dense(32, activation='relu')
        self.layer3 = tf.keras.layers.Dense(action_shape)

    def call(self, state):
        x = self.layer1(state)
        x = self.layer2(x)
        return self.layer3(x)

In [7]:
# Initialize the DQN model and target model
state_shape = (X_train.shape[1], )  # Shape of one state
action_shape = 3  # Number of actions: Buy, Hold, Sell
dqn_model = DQN(state_shape, action_shape)

# Compile the model
dqn_model.compile(optimizer='adam', loss='mean_squared_error')

In [8]:
# Load model if resuming training
checkpoint_path = "dqn_model_checkpoint"
if tf.train.latest_checkpoint(checkpoint_path):
    dqn_model.load_weights(checkpoint_path)

In [9]:
class TradingEnvironment:
    def __init__(self, data, initial_balance=10000):
        self.data = data
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.current_step = 0
        self.current_price = self.data[self.current_step, 0]  # Assuming the price is in the first column
        self.position = None

    def step(self, action):
        self.current_step += 1
        self.current_price = self.data[self.current_step, 0]  # Assuming the price is in the first column

        reward = 0
        done = self.current_step == len(self.data) - 1

        # 0: Buy, 1: Hold, 2: Sell
        if action == 0 and self.position is None:  # Buy
            self.position = self.current_price
        elif action == 2 and self.position is not None:  # Sell
            reward = self.current_price - self.position
            self.balance += reward
            self.position = None

        next_state = self.data[self.current_step]
        return next_state, float(reward), done  # Making sure reward is a float

    def reset(self):
        self.balance = self.initial_balance
        self.current_step = 0
        self.current_price = self.data[self.current_step, 0]  # Assuming the price is in the first column
        self.position = None
        return self.data[self.current_step]


In [10]:
# Initialize environment (modify your existing TradingEnvironment class accordingly)
env = TradingEnvironment(data=X_train)

In [ ]:
# Initialize replay buffer
replay_buffer = []

# Training parameters
n_episodes = 1000
batch_size = 32
gamma = 0.99
epsilon = 0.1

# Training loop
for episode in range(1, n_episodes + 1):
    state = env.reset()
    done = False

    while not done:
        if random.random() < epsilon:
            action = random.choice([0, 1, 2])
        else:
            action = np.argmax(dqn_model.predict(state.reshape(1, -1)))

        next_state, reward, done = env.step(action)
        replay_buffer.append((state, action, reward, next_state, done))

        if len(replay_buffer) > batch_size:
            minibatch = random.sample(replay_buffer, batch_size)
            states, actions, rewards, next_states, dones = zip(*minibatch)
            states, actions, rewards, next_states, dones = np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones)

            q_values = dqn_model.predict(states)
            next_q_values = dqn_model.predict(next_states)
            next_q_value = np.max(next_q_values, axis=1)

            target_q_values = q_values
            target_q_values[np.arange(batch_size), actions] = rewards + (gamma * next_q_value * (1 - dones))

            dqn_model.fit(states, target_q_values, epochs=1, verbose=0)

        state = next_state

    # Save model checkpoint
    dqn_model.save_weights(checkpoint_path)

1/1 [==============================] - 0s 32ms/step


In [ ]:
# Test model
env = TradingEnvironment(data=X_test)
state = env.reset()
done = False
total_reward = 0
num_profitable_trades = 0
num_loss_making_trades = 0

while not done:
    action = np.argmax(dqn_model.predict(state.reshape(1, -1)))
    next_state, reward, done = env.step(action)
    total_reward += reward
    if reward > 0:
        num_profitable_trades += 1
    elif reward < 0:
        num_loss_making_trades += 1
    state = next_state

print(f"Total reward from test set: {total_reward}")
print(f'Number of Profitable Trades: {num_profitable_trades}')
print(f'Number of Loss-making Trades: {num_loss_making_trades}')

In [ ]:
import ccxt  # Library to interact with crypto exchange APIs

# Initialize Binance API
binance = ccxt.binance({
    'apiKey': 'YOUR_API_KEY',
    'secret': 'YOUR_API_SECRET'
})

import pandas as pd
import numpy as np
from talib import RSI, BBANDS, MACD


In [ ]:
# Initialize a DataFrame to keep a rolling window of data
rolling_data = pd.DataFrame()

# Function to fetch real-time BTC/USD daily data (OHLCV)
def fetch_real_time_data():
    ohlcv = binance.fetch_ohlcv('BTC/USDT', '1d')  # 1d specifies daily data
    latest_data = ohlcv[-1]  # Get the latest data
    return latest_data  # You may want to select specific fields

# Function to apply technical indicators (modify this based on your specific needs)
def apply_technical_indicators(data):
    # Apply RSI, Bollinger Bands, MACD
    data['rsi'] = talib.RSI(data['close'], timeperiod=14)
    data['upper_band'], data['middle_band'], data['lower_band'] = talib.BBANDS(data['close'], timeperiod=20)
    data['macd'], data['signal'], _ = talib.MACD(data['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    return data

# Implement paper trade (modify this to your real-world data)
real_world_data = fetch_real_time_data()  # Fetch real-world data
real_world_data = apply_technical_indicators(real_world_data)  # Apply technical indicators
paper_env = TradingEnvironment(data=real_world_data)

state = paper_env.reset()
done = False

while not done:
    action = np.argmax(dqn_model.predict(state.reshape(1, -1)))
    next_state, reward, done = paper_env.step(action)
    state = next_state

print("Finished paper trading.")


In [ ]:
# Initialize a DataFrame to keep a rolling window of data
rolling_data = pd.DataFrame()

def fetch_real_time_data(window_size=50):
    global rolling_data

    # Fetch latest OHLCV data
    ohlcv = binance.fetch_ohlcv('BTC/USDT', '1d')
    latest_data = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume']).tail(1)

    # Add to rolling data
    rolling_data = pd.concat([rolling_data, latest_data]).tail(window_size)

    # Calculate RSI
    rolling_data['rsi'] = RSI(rolling_data['close'], timeperiod=14)

    # Calculate Bollinger Bands
    upper, middle, lower = BBANDS(rolling_data['close'], timeperiod=20)
    rolling_data['upper_band'] = upper
    rolling_data['middle_band'] = middle
    rolling_data['lower_band'] = lower

    # Calculate MACD
    macd, signal, _ = MACD(rolling_data['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    rolling_data['macd'] = macd
    rolling_data['signal'] = signal

    # Assuming your model uses these features: ['close', 'rsi', 'upper_band', 'middle_band', 'lower_band', 'macd', 'signal']
    latest_data_with_indicators = rolling_data[['close', 'rsi', 'upper_band', 'middle_band', 'lower_band', 'macd', 'signal']].tail(1).values

    return latest_data_with_indicators


class RealTimeTradingEnvironment(TradingEnvironment):
    def __init__(self, initial_balance=10000):
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.current_step = 0
        self.current_price = fetch_real_time_data()
        self.position = None

    def step(self, action):
        self.current_step += 1
        self.current_price = fetch_real_time_data()  # Fetching real-time data

        reward = 0
        done = False  # The paper trading loop will continue until you stop it

        # 0: Buy, 1: Hold, 2: Sell
        if action == 0 and self.position is None:  # Buy
            self.position = self.current_price
        elif action == 2 and self.position is not None:  # Sell
            reward = self.current_price - self.position
            self.balance += reward
            self.position = None

        next_state = self.current_price
        return next_state, float(reward), done

    # Rest of the functions remain the same

# Initialize the real-time environment
paper_env = RealTimeTradingEnvironment()
state = paper_env.reset()
done = False

# Paper trading loop
while not done:
    action = np.argmax(dqn_model.predict(state.reshape(1, -1)))
    next_state, reward, done = paper_env.step(action)
    state = next_state
    print(f"Action taken: {action}, Reward received: {reward}, Balance: {paper_env.balance}")

print("Finished paper trading.")
